In [1]:
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from time import sleep
from datetime import datetime
import pytz
import json

In [2]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))  # Add parent directory to Python path
from driver import Bot

from colorama import Fore, Style
from selenium.webdriver.common.keys import Keys 

# Initialize the bot
bot = Bot()

# Test login and chat counting functionality
print(Fore.GREEN + "Starting WhatsApp Web test..." + Style.RESET_ALL)
# bot.login_and_count_chats()

# Keep the session open for manual testing
# When done testing, you can close the driver with:
# bot.quit_driver()

Starting WhatsApp Web test...


In [3]:
bot.login('+55')

Attempting to load WhatsApp Web...
Logged in successfully!


TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType

In [25]:
#bot.click_first_chat_and_scroll()
# GET ALL CONTACTS NAMES FROM CHATS LIST
# Localizar a lista de chats
from config import GROUP_ANALYZE


all_chats = set()  # Usar set para evitar duplicatas
last_seen_name = None  # Nome do último item visível na lista
# chat_list_element = bot.driver.find_element(By.ID, "pane-side")

while True:
    # Encontrar os chats visíveis
    chat_items = bot.driver.find_elements(By.XPATH, "//div[@aria-label='Lista de conversas']//div[@role='listitem']")
    if len(chat_items) == 0:
        chat_items = bot.driver.find_elements(
            By.XPATH, "//div[@aria-label='Chat list']//div[@role='listitem']")

    # chat_items = bot.driver.find_elements(By.CLASS_NAME, "x10l6tqk")
    # new_last_seen_name = None
    
    # se todos os chats já foram coletados, sair do loop, checa todos os no
    already_collected_chats = []
    # para teste somente 1 chat
    # chat_items = [chat_items[0], chat_items[1], chat_items[2],
    #                chat_items[3], chat_items[4], chat_items[5]]
    # chat_items = [chat_items[0],chat_items[5]]
    for chat in chat_items:
        try:
            # Captura o nome do chat e outras informações
            name = chat.find_element(By.XPATH, ".//span[@dir='auto']").text
            print(f'name: {name}')
            # Adicionar informações únicas ao conjunto
            if name not in all_chats:
                all_chats.add(f"{name}")
                chat.click()
                sleep(2)
                # get all messages and save in a file
                print(f'getting messages from {name}')
                # messages = get_all_messages()
                
                chat.click()
                sleep(2)

                # Click on Profile details
                profile_details = bot.driver.find_element(
                    By.XPATH, "//div[@title='Profile details']")
                profile_details.click()
                sleep(2)
                
                # Check if the chat is a group
                group_text = bot.driver.find_elements(
                    By.XPATH,
                    "//section[contains(@class, 'x2lah0s')]//span[contains(text(), 'Group')]"
                )
                if len(group_text) > 0:
                    if GROUP_ANALYZE:
                        # Do something
                        print(f'group_text: {group_text[0].text}')
                    else:
                        print(f'not analize group')
                        # sair do loop for e pular para o proximo chat
                        continue
                
                phone_number = None
                # Check if the chat is a business
                business_text = bot.driver.find_elements(
                    By.XPATH,
                    "//div[contains(@class, 'x1iyjqo2') and contains(text(), 'This is a business account')]"
                )
                if len(business_text) > 0:
                    phone_element = bot.driver.find_elements(
                        By.XPATH,
                        "//span[@class='x1rg5ohu x13faqbe _ao3e selectable-text copyable-text']"
                    )
                    
                else:
                    print(f'not business')
                    phone_element = bot.driver.find_elements(
                        By.XPATH,
                        "//section[contains(@class, 'x2lah0s')]//span[@class='x1jchvi3 x1fcty0u x40yjcy']"
                    )
                    
                if len(phone_element) > 0:
                    phone_number = phone_element[0].text
                    print(f'phone_number: {phone_number}')
                    
                # close profile details
                close_button = bot.driver.find_elements(
                    By.XPATH,
                    "//span[@data-icon='x']"
                )
                if len(close_button) > 0:
                    close_button[0].click()
                    sleep(2)
                else:
                    print(f'close_button not found')
                        
           
        except Exception as e:
            print(f"Erro ao processar um chat: {e}")
    break

print(all_chats)
print(len(all_chats))

name: Gustavo Bromer
getting messages from Gustavo Bromer
not business
phone_number: +55 48 9639-7142
name: Thata Pereira
getting messages from Thata Pereira
not business
phone_number: +55 11 93461-7916
name: Bruno Koerich
getting messages from Bruno Koerich
not business
phone_number: +55 48 8469-5280
name: Equipe Contador Direto
getting messages from Equipe Contador Direto
phone_number: Equipe Contador Direto
name: Casa da árvore, só vai quem trepa
getting messages from Casa da árvore, só vai quem trepa
not analize group
name: +55 48 9662-4847
getting messages from +55 48 9662-4847
phone_number: +55 48 9662-4847
name: Familia os 4
getting messages from Familia os 4
not analize group
name: #2 Analises do Pit
getting messages from #2 Analises do Pit
not analize group
name: Bel Isabel Faillace
getting messages from Bel Isabel Faillace
not business
phone_number: +55 48 9656-5530
name: Eu 51
getting messages from Eu 51
phone_number: Anderson LT
name: Machado Surf Treino!!!
getting messages

In [5]:
# SEARCH THE CONTACT
message_input = bot.driver.find_element(
    By.XPATH, "//div[contains(@class, 'x1hx0egp')][@role='textbox']")
message_input.click()
first_contact = next(iter(all_chats))
message_input.send_keys(first_contact)
message_input.send_keys(Keys.ENTER)
sleep(2)
